### Notebook to generate DroneRF Features

In [1]:
import os
import numpy as np
from numpy import sum,isrealobj,sqrt
from numpy.random import standard_normal
import os
from sklearn.model_selection import train_test_split
from spafe.features.lfcc import lfcc
import spafe.utils.vis as vis
from scipy.signal import get_window
import scipy.fftpack as fft
from scipy import signal
import matplotlib.pyplot as plt
from datetime import date
from tqdm import tqdm

from loading_functions import *
from file_paths import *

In [2]:
def save_array(folder_path, feat, BI, DRONES, MODES, feat_name, seg_i):
    Xs_arr = np.array(feat)
    
    # labels
    y_bi_arr = np.array(BI)
    y_drones_arr = np.array(DRONES)
    y_modes_arr = np.array(MODES)

    data_save = {'feat': Xs_arr, 'bi': y_bi_arr, 'drones': y_drones_arr, 'modes': y_modes_arr}
#     return data_save

#     #Save data
    date_string = date.today()
    fp = folder_path+feat_name+"_"+str(n_per_seg)+"_"+str(seg_i)
    print(fp)
    np.save(fp, data_save)

In [3]:
# Dataset Info
main_folder = droenrf_raw_path
t_seg = 20
Xs_arr, ys_arr, y4s_arr, y10s_arr = load_dronerf_raw(main_folder, t_seg)
fs = 40e6 #40 MHz

print('length of X:', len(Xs_arr), 'length of y:', len(ys_arr))

diff 5 file name: 00000L_13.csv
length of X: 2712 length of y: 2712


In [4]:
Xs_arr.shape[0]

2712

In [5]:
n_per_seg = 256 # length of each segment (powers of 2)
win_type = 'hamming' # make ends of each segment match
high_low = 'H' #'L', 'H' # high or low range of frequency

# data saving folders
features_folder = dronerf_feat_path
date_string = date.today()
# folder naming: ARR_FEAT_NFFT_SAMPLELENGTH
arr_spec_folder = "ARR_SPEC_"+high_low+'_'+str(n_per_seg)+"_"+str(t_seg)+"/"
arr_psd_folder = "ARR_PSD_"+high_low+'_'+str(n_per_seg)+"_"+str(t_seg)+"/"

if high_low == 'H':
    i_hl = 0
elif high_low == 'L':
    i_hl = 1

In [6]:
# check if this set of parameters already exists
existing_folders = os.listdir(features_folder)
n_t_combo = high_low+'_'+str(n_per_seg)+"_"+str(t_seg)
if any([n_t_combo in s for s in existing_folders]):
    print('Features Already Exist - Do Not Generate')
else:
    os.mkdir(features_folder+arr_spec_folder)
    os.mkdir(features_folder+arr_psd_folder)

In [7]:
n_parts = 25 # process the data in 10 parts so memory doesn't overwhelm

indices_ranges = np.split(np.array(range(len(Xs_arr))), n_parts) 
for i in range(n_parts):
    BILABEL = []
    DRONELABEL = []
    MODELALBEL = []
    F_PSD = []
    F_SPEC = []
    ir = indices_ranges[i]
    for j in tqdm(range(len(ir))):
        d_real = Xs_arr[ir[j]][i_hl]
        # calculate PSD
        fpsd, Pxx_den = signal.welch(d_real, fs, window=win_type, nperseg=n_per_seg)
        F_PSD.append(Pxx_den)

        # calculate spectrogram
        fspec, t, Sxx = signal.spectrogram(d_real, fs, window=win_type, nperseg=n_per_seg)
        F_SPEC.append(Sxx)
        
        # Labels
        BILABEL.append(ys_arr[ir[j]])
        DRONELABEL.append(y4s_arr[ir[j]])
        MODELALBEL.append(y10s_arr[ir[j]])

    save_array(features_folder+arr_spec_folder, F_SPEC, BILABEL, DRONELABEL, MODELALBEL, 'SPEC', i)
    save_array(features_folder+arr_psd_folder, F_PSD, BILABEL, DRONELABEL, MODELALBEL, 'PSD', i)

ValueError: array split does not result in an equal division

In [ ]:
t_seg